In [ ]:
# import pandas as pd
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# import matplotlib.pyplot as plt
# import numpy as np
# from scipy.stats import norm
# from keras.layers import Input, Dense, Lambda, Flatten, Reshape
# from keras.layers import Conv1D
# from keras.models import Model
# from keras import backend as K
# from sklearn.utils import resample
# 
# n_features = 4
# n_runs = 200
# labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_len_labels.csv")
# labels = labels.drop(columns="Unnamed: 0") 
# labels = np.array(labels)
# max_len = 0
# 
# 
# 
# def prepare_training(path, n_runs):
#     def closest_4(n, m):
#         q = n / m
#         n1 = m * q
#         if (n * m) > 0:
#             n2 = m * (q + 1)
#         else:
#             n2 = m * (q - 1)
#         if abs(n-n1) < abs(n-n2):
#             return int(n1)
#         return int(n2)
#     
#     
#     def extend_line(run, max_len):
#         difference = abs(len(run) - max_len)
#         extension = np.array([run[-1]]*difference)
#         if difference != 0:
#             run = np.vstack([run, extension])
#         return run
#     
#     def get_max_len(sequence_list):
#         max_len = 0
#         min_len = 1000
#         for seq in sequence_list:
#             if len(seq) > max_len:
#                 max_len = len(seq)
#             if len(seq) < min_len:
#                 min_len = len(seq)
#         return max_len, min_len
#     
#     def construct_matrix(sequence_list):
#         max_len, min_len = get_max_len(sequence_list)
#         print(max_len)
#         len = closest_4(max_len,4)
#         len = 620
#         train_matrix = np.zeros(shape=(n_runs, len, n_features))
#         for index, run in enumerate(sequence_list):
#             line = extend_line(run, len)
#             train_matrix[index] = line
#         return train_matrix
#         
#         
#     def stadard_sequences(seqs):
#         for i, seq in enumerate(seqs):
#             seqs[i] = MinMaxScaler(feature_range=[0, 1]).fit_transform(seq)
#         return seqs       
#     
#     
#     def read_sequences():
#         run_list_mix = []
#         for index in range(n_runs):
#             run_csv = pd.read_csv(path+str(index))
#             run_csv = run_csv.drop(columns=['Unnamed: 0'])
#             run_list_mix.append(run_csv)
#         stands = stadard_sequences(run_list_mix)
#         padded_matrix = construct_matrix(stands)
#         return padded_matrix
#     
#     return read_sequences()
#     
# 
# train_matrix = prepare_training("Mix_sequences_var_length/run^", n_runs=n_runs) 
# print(train_matrix.shape)


615


In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D
from keras.models import Model
from keras import backend as K
from sklearn.utils import resample

n_features = 4
latent_dim = 10
n_runs = 300
max_len = 0

def prepare_training(path, n_runs):
    labels = []
    def closest_4(n, m):
        q = n / m
        n1 = m * q
        if (n * m) > 0:
            n2 = m * (q + 1)
        else:
            n2 = m * (q - 1)
        if abs(n-n1) < abs(n-n2):
            return int(n1)
        return int(n2)
    
    
    def extend_line(run, max_len):
        difference = abs(len(run) - max_len)
        extension = np.array([run[-1]]*difference)
        if difference != 0:
            run = np.vstack([run, extension])
        return run
    
    def get_max_len(sequence_list):
        max_len = 0
        min_len = 1000
        for seq in sequence_list:
            if len(seq) > max_len:
                max_len = len(seq)
            if len(seq) < min_len:
                min_len = len(seq)
        return max_len, min_len
    
    def construct_matrix(sequence_list):
        max_len, min_len = get_max_len(sequence_list)
        print(max_len)
        len = closest_4(max_len,4)
        len = 620
        train_matrix = np.zeros(shape=(n_runs, len, n_features))
        for index, run in enumerate(sequence_list):
            line = extend_line(run, len)
            train_matrix[index] = line
        return train_matrix
        
        
    def stadard_sequences(seqs):
        for i, seq in enumerate(seqs):
            seqs[i] = MinMaxScaler(feature_range=[0, 1]).fit_transform(seq)
        return seqs       
    
    
    def order_runs_by_len(runs):
        runs.sort(key=len)
        for r in runs:
            labels.append(r['Choice'][0])
        for i,r in enumerate(runs):
            runs[i] = runs[i].drop(columns=['Choice'])
        return runs, labels
    
    def read_sequences():
        run_list_mix = []
        for index in range(n_runs):
            run_csv = pd.read_csv(path+str(index))
            run_csv = run_csv.drop(columns=['Unnamed: 0'])
            run_list_mix.append(run_csv)
        run_list_ordered, labels = order_runs_by_len(run_list_mix)
        stands = stadard_sequences(run_list_ordered)
        padded_matrix = construct_matrix(stands)
        return padded_matrix, labels
    
    return read_sequences()

train_matrix, labels = prepare_training("Mix_sequences_var_length/run^", n_runs=n_runs) 
labels = np.array(labels)



Using TensorFlow backend.


607


In [ ]:
import keras
from keras import objectives
from keras.layers import Input, LSTM, RepeatVector, Conv2DTranspose, MaxPooling1D, UpSampling1D, AveragePooling1D
from keras.losses import mse
from keras.models import Model


filters = 40
intermediate_dimension = 50 
latent_dim = 10


def Conv1DTranspose(input_tensor, filters, kernel_size,strides=1, padding='same'):
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1),
                            activation='relu',
                            strides=strides, padding='same')(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x


def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim), mean=0., stddev=0.000001)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon
# 
# 
# def repeat(x):
#     steps_matrix = K.ones_like(x[0][:, :, :1])
#     
#     latent_matrix = K.expand_dims(x[1], axis=1)
#     return K.batch_dot(steps_matrix, latent_matrix)


def create_vae():
    inputs = Input(shape=(train_matrix.shape[1], n_features), name='Vae_input')
    x = inputs
    
    for i in range(4):
        x = Conv1D(filters=filters,
                   kernel_size=20,
                   activation='relu',
                   padding='same')(x)
        x = MaxPooling1D(pool_size=2)(x)


            
    encoder_outputs, state_h, state_c = LSTM(latent_dim, return_state=True)(x)
    encoder_states = [state_h, state_c]     
    
    z_mean = Dense(latent_dim, name='z_mean',)(state_h)
    z_log_var = Dense(latent_dim, name='z_log_var')(state_h)
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
    
    encoder = Model(inputs, [z_mean, z_log_var, z, state_h, state_c], name='encoder')
    # 
    #latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
    # x_dec = Dense(shape[1]*shape[2])(z)
    # x_dec = Reshape((shape[1], shape[2]))(x_dec)
    
    decoder_inputs = Input((train_matrix.shape[1], n_features), name='dec_input')
    decoder_lstm = LSTM(latent_dim, return_sequences=True)
    dec_lstm = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    # x_dec = Dense(shape[1]*shape[2])(dec_lstm)
    # x_dec = Reshape((shape[1], shape[2]))(x_dec)
    
    x_dec = dec_lstm
    for i in range(2):
        x_dec = Conv1DTranspose(input_tensor=x_dec,
                            filters=filters,
                            kernel_size=20,
                            padding='same')
        #x_dec = UpSampling1D(size=2)(x_dec)
        
    output = Dense(n_features)(x_dec)
    
    # decoder = Model(latent_inputs, output)
    # 
    # output = decoder(encoder.outputs[2])
    # 
    
    xent_loss = objectives.mean_squared_error(K.flatten(inputs), K.flatten(output))
    kl_loss = 0.5 * K.sum(K.exp(z_log_var) + K.square(z_mean) - 1. - z_log_var, axis=1)
    loss = K.mean(xent_loss+kl_loss)
    vae = Model([inputs, decoder_inputs], output, name='vae')
    vae.add_loss(loss)
    vae.compile(optimizer='rmsprop')
    vae.summary()
    
    return vae, encoder

# 
# def create_ae():
#     inputs = Input(shape=(train_matrix.shape[1], n_features))
#     x = inputs
#     for i in range(2):
#         x = Conv1D(filters=filters, kernel_size=20,
#                    activation='relu',
#                    padding='same')(x)
#         x = AveragePooling1D(pool_size=2)(x)
#     shape = K.int_shape(x)   
#     print(shape)
#     x,_,_ = LSTM(intermediate_dimension, return_state=True)(x)
#     #x = Flatten()(x)
#     encoded = Dense(latent_dim)(x)
#     #x = Lambda(repeat)([before_flattening, encoded])
# 
#     latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
#     x = Dense(shape[1]*shape[2])(latent_inputs)
#     x = Reshape((shape[1],shape[2]))(x)
#     
#     x = LSTM(units=K.int_shape(x)[2],return_sequences=True)(x)
#     for i in range(2):
#         x = Conv1DTranspose(input_tensor=x, filters=filters,
#                             kernel_size=20, padding='same')
#         x = UpSampling1D(size=2)(x)
# 
#     #decoded = LSTM(n_features, return_sequences=True)(x)
# 
#     output = Dense(n_features)(x)
# 
#     encoder = Model(inputs, encoded)
#     encoder.summary()
#     decoder = Model(latent_inputs, output)
#     decoder.summary()
#     output = decoder(encoder.output)
#     sequence_autoencoder = Model(inputs, output)
#     #sequence_autoencoder.summary()
#     sequence_autoencoder.compile(optimizer='adam', 
#                                  loss='mse')
#     return sequence_autoencoder, encoder, shape


model, encoder = create_vae()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Vae_input (InputLayer)          (None, 620, 4)       0                                            
__________________________________________________________________________________________________
conv1d_62 (Conv1D)              (None, 620, 40)      3240        Vae_input[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_44 (MaxPooling1D) (None, 310, 40)      0           conv1d_62[0][0]                  
__________________________________________________________________________________________________
conv1d_63 (Conv1D)              (None, 310, 40)      32040       max_pooling1d_44[0][0]           
__________________________________________________________________________________________________
max_poolin

In [62]:
from keras.callbacks import ModelCheckpoint

def train():
    
    print(train_matrix.shape)
    model.fit([train_matrix,train_matrix], epochs=15, verbose=1)
    model.save_weights("Models/Weights/AE_CONV_LSTM_Diff_len_dist_MATRIX_LEN.hdf5")


train()


(300, 620, 4)


Epoch 1/15


 32/300 [==>...........................] - ETA: 5:36 - loss: 0.3760

 64/300 [=====>........................] - ETA: 2:38 - loss: 2.8518

 96/300 [========>.....................] - ETA: 1:37 - loss: 1.9492

...

 96/300 [========>.....................] - ETA: 20s - loss: 0.0289

128/300 [===========>..................] - ETA: 17s - loss: 0.0265

160/300 [===============>..............] - ETA: 15s - loss: 0.0250

192/300 [==================>...........] - ETA: 13s - loss: 0.0241

224/300 [=====================>........] - ETA: 9s - loss: 0.0235 

In [60]:

def return_mask(num, labs):
    arg = np.squeeze(np.argwhere(labs == num))
    return arg

masks = [return_mask(num, np.array(labels)) for num in range(0, 9)]

from sklearn.decomposition import PCA


encodings = encoder.predict(train_matrix)

#enc_mean, enc_var, z_enc = encodings[0], encodings[1], encodings[2]
enc_mean, enc_var, z_enc, state_h, state_c = encodings[0], encodings[1], encodings[2], encodings[3], encodings[4]

print(enc_mean.shape, enc_var.shape, z_enc.shape)


from mpl_toolkits.mplot3d import Axes3D  



def plot_pca(title, i): 
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    markers = ['o', 'o', 'o', 'o', '^', '^', '^', '^', '^', '^']
    for index, mask in enumerate(masks):
        
        ax.scatter(principalComponents[:, 0][mask], 
                   principalComponents[:, 1][mask],  
                   principalComponents[:, 2][mask], marker=markers[index])
    # for mask in unseen_mask:
    #     ax.scatter(unseen_encoding[0][:,0][mask],
    #                unseen_encoding[0][:,1][mask],
    #                unseen_encoding[0][:,2][mask])
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    
    for mask in masks:
        plt.scatter(x=principalComponents[:, 0][mask], 
                    y=principalComponents[:, 1][mask],
                    alpha=0.5)
    # for mask in unseen_mask:
    #     plt.scatter(unseen_encoding[0][:,0][mask],
    #            unseen_encoding[0][:,1][mask])
    #     
        #break
    
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()


enc_list = [enc_mean, enc_var, z_enc, state_h, state_c]
titles = ["MEAN","LOG_VAR","SAMPLED", 'state_h', 'state_c']
for i,enc in enumerate(enc_list):
    scaler = StandardScaler()
    enc_input = scaler.fit_transform(enc) 
    pca = PCA(3)
    principalComponents = pca.fit_transform(enc_input)
    print(principalComponents.shape)
    print(pca.explained_variance_ratio_)
    plot_pca('Sequences'+titles[i], 0)
    
    # principalComponents = enc
    # plot_pca('Sequences_Not_Pca'+titles[i], 0)
    



(300, 10) (300, 10) (300, 10)
(300, 3)
[0.6983651  0.23717883 0.04107158]


(300, 3)
[0.74839777 0.22248009 0.01759055]


(300, 3)
[0.6939893  0.23281148 0.04287714]


(300, 3)
[0.6719166  0.18468654 0.10815211]


(300, 3)
[0.6719133  0.18468542 0.10815584]


In [75]:

unseen_sequences_matrix, unseen_labs = prepare_training("Mix_sequences_var_length/run_unseen^", 
                                           n_runs=300)
unseen_mask = [return_mask(num, np.array(unseen_labs)) for num in range(0, 9)]

unseen_encoding = encoder.predict(unseen_sequences_matrix)

628


ValueError: could not broadcast input array from shape (636,4) into shape (620,4)

In [116]:
reconstruction = model.predict([train_matrix, train_matrix])
print(reconstruction.shape)
#RECONSTRUCTION
def reconstruct_sequence(seq_index):
    run = train_matrix[seq_index]
    #mask_seq = np.squeeze(np.argwhere(np.mean(run, axis=1) != 0))
    rec_run = reconstruction[seq_index]#[mask_seq]
    #print(rec_run[:,0])
    df = pd.DataFrame(rec_run[:-15], columns=["Sin", "Cosin", "Lat", "Lon"])
    df_original = pd.DataFrame(run[:-15], columns=["Sin", "Cosin", "Lat", "Lon"])
    plt.plot(df_original['Lon'], df_original['Lat'])
    plt.plot(df['Lon'], df['Lat'])
    plt.show()

for i in range(len(train_matrix)):
    reconstruct_sequence(i)
    if i == 5:
        break


(300, 620, 4)


In [3]:

model.load_weights("Models/Weights/VAE_CONV_LSTM_Diff_len_dist_MATRIX_LEN.hdf5")


In [32]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)

tsne_obj= tsne.fit_transform(enc_input)

print(tsne_obj.shape)

for mask in masks:
    plt.scatter(x=tsne_obj[:, 0][mask], 
                y=tsne_obj[:, 1][mask],
                alpha=0.5)
plt.show()


(200, 2)


In [146]:
#RECONSTRUCTION
def reconstruct_sequence(seq_index):
    run = train_matrix[seq_index]
    #mask_seq = np.squeeze(np.argwhere(np.mean(run, axis=1) != 0))
    
    reconstr_run = model.predict(np.reshape(run, (1, run.shape[0], run.shape[1])))
    reconstr_run = np.reshape(reconstr_run, (run.shape[0], run.shape[1]))
    reconstr_run = reconstr_run
    df = pd.DataFrame(reconstr_run, columns=["Timestep","Sin", "Cosin", "Lat", "Lon"])
    plt.plot(df['Lon'], df['Lat'])
    plt.show()

for i in range(len(train_matrix)):
    reconstruct_sequence(i)
    if i > 5:
        break
    



In [23]:

reconstruction = model.predict(train_matrix)
reconstruction_unseen = model.predict(unseen_sequences_matrix)


#RECONSTRUCTION ERROR
def get_reconstructed_matrix(input_matrix, reconstrut):
    return_matrix = np.zeros(shape=input_matrix.shape)
    for i,run in enumerate(input_matrix):
        mask_seq = np.squeeze(np.argwhere(np.mean(run, axis=1) != 0))
        return_matrix[i][mask_seq] = reconstrut[i][mask_seq]    
    return reconstrut


train_error = abs(train_matrix-get_reconstructed_matrix(train_matrix,reconstruction))
unseen_runs_error = abs(unseen_sequences_matrix-get_reconstructed_matrix(train_matrix,reconstruction_unseen))

train_error_avg = np.mean(train_error, axis=2)
unseen_error_avg = np.mean(unseen_runs_error, axis=2)
train_error_avg = np.mean(train_error_avg, axis=1)
unseen_error_avg = np.mean(unseen_error_avg, axis=1)
print(train_error_avg.shape, unseen_error_avg.shape)

for mask in masks:
    plt.scatter(np.linspace(1,200,200)[mask],train_error_avg[mask])
plt.title('ERROR ON TRAIN')
plt.show()

for mask in unseen_mask:
    plt.scatter(np.linspace(1,200,200)[mask],unseen_error_avg[mask])
plt.title('ERROR ON Unseen')
plt.show()

(200,) (200,)


In [18]:
import ipyvolume as ipv
import numpy as np
x, y, z = unseen_encoding[:,0], unseen_encoding[:,1], unseen_encoding[:,2]

for mask in unseen_mask:
    ipv.scatter(x[mask], y[mask], z[mask], size=0.3, marker="sphere")
ipv.show()

[[1.69657601e-04 2.90231335e-03 2.66825905e-01 5.29824561e-01
  8.35837121e-01]
 [1.69657601e-04 2.90231335e-03 3.53738663e-01 5.49122807e-01
  7.10645313e-01]
 [9.93304461e-03 5.59014899e-02 3.40350877e-01 7.50188854e-01
  9.98851458e-01]
 ...
 [0.00000000e+00 6.14035088e-02 3.62061103e-01 4.36881822e-01
  1.00000000e+00]
 [4.80214197e-03 2.02577052e-01 6.96491228e-01 7.76227370e-01
  9.99448401e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]]


In [6]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available(), torch.backends.cudnn.enabled)


False

True

In [7]:
from __future__ import print_function
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.3990, 0.5784, 0.8855],
        [0.1293, 0.7703, 0.6091],
        [0.2355, 0.0893, 0.3447],
        [0.3635, 0.8326, 0.8823],
        [0.9676, 0.3495, 0.6277]])

In [8]:
torch.cuda.is_available()

False